In [31]:
# Driver

# Küldjünk valami csomagot a workernek

import numpy as np
import requests
import codecs
import pickle
import joblib
import json
import os
from warnings import simplefilter
from sklearn import datasets
from sklearn.neural_network import MLPRegressor
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

# beállítunk néhány paraméter ami amit később használunk
# ezeket az értékeket sajnos a Driver oldalon is ki kell számolnunk, ha úgy csináljuk, hogy a modelt küldjük át
# fontos, hogy számos paraméter szinkronizálva legyen, különben ami model itt előáll az a Worker oldalon eltörik
arch = (2,2)                                # <-- nn(arch)
window = 21                                 # <-- bementi változók száma
nRowsRead = 9999                            # <-- ez lehet alapján csinálunk modelt, de lehet más is mint odaát
threshold = -1.0                         # <-- a trader parametére

# elő kell állítani a túloldali adatsorral azonos szart np.[970, 20] és np[970]
x_train = np.ones((970, window))
y_train = np.zeros(970)
print(x_train.shape)
print(y_train.shape)
clf = MLPRegressor(hidden_layer_sizes=arch, max_iter=3, shuffle=False, activation='tanh', random_state=1)
clf.fit(x_train, y_train)

# ---------------------------------------------------------------------------
# Ugyan ez a Joblib használatával (ez valami filét is csiál rendesen a diskre)
# from joblib import dump, load
joblib.dump(clf, 'model.joblib')         # <-- elmenjük
clf = joblib.load('model.joblib')        # <-- betöltjük

# ---------------------------------------------------------------------------
# nem a pickle-t használom, hanem a joblibet, de itt hagyom
# s = pickle.dumps(clf)     # <-- csinálunk belőe bytot
# print(type(s))            # <-- ez byte valahogy biztosan át lehet küldeni.
# clf = pickle.loads(s)    # <-- betöltjük
# clf.predict(x_train)


# ---------------------------------------------------------------------------














# -----------------------------------
a = np.array([[1,2,3],[4,5,6]])
lists = a.tolist()
data = json.dumps(lists)
print(data)

# Ezek majd mennek a kukába
# -----------------------------------

worker_ip = '192.168.0.247'


# Egyik módszer
data_json = json.dumps(lists)
payload = {'json_payload': data_json, 'host': '1', 'individual': '1'}
r = requests.get('http://192.168.0.247:8080', data=payload)
print(r)

# Másik módszer
r = requests.post('http://192.168.0.247:8080/model', json={"temperature": "24.3"})
print(r)
# 405 mert még nincs rajta a post request a workeren


# ---------------------------------------------------------------------------


# Setuppolni kell a paramétereket
request_params = 'nRowsRead=' + (str)(nRowsRead) + '&' + 'window=' + (str)(window) + '&' + 'threshold=' + (str)(threshold)
print(request_params)
# resp = requests.get('http://192.168.0.247:8080/setup?nRowsRead=2998&window=20&threshold=-1000')
resp = requests.get('http://192.168.0.247:8080/setup?' + request_params)

print('setup      ', resp)

# Ezzel simán meghívjuk a Worker INITIALIZE REST API Végpontját
resp = requests.get('http://192.168.0.247:8080/initilaize')
print('initialize ', resp)

# Ezzel a módszerrel lehet átküldeni neki a joblib model filét
uploadResultUrl = 'http://192.168.0.247:8080/uploader'
filePath = "model.joblib"
fileFp = open(filePath, 'rb')
fileInfoDict = {
    "file": fileFp,
}
resp = requests.post(uploadResultUrl, files=fileInfoDict)
print('uploader   ', resp)                                # <-- ez nagyon jó ha <Response [200]> mert akkor átment a file


# Átküldök egy értéket a Workernek
resp = requests.get('http://192.168.0.247:8080/testpoint?value=123456789')
print('testpoint  ', resp)


(970, 21)
(970,)
[[1, 2, 3], [4, 5, 6]]
<Response [200]>
<Response [404]>
nRowsRead=9999&window=21&threshold=-1.0
setup       <Response [200]>
initialize  <Response [200]>
uploader    <Response [200]>
testpoint   <Response [200]>
